In [21]:
import yaml, subprocess, os, json, sys
from datetime import datetime
import pandas as pd
import numpy as np
import ast, redis

WATCHDOG_ADDRESS = "192.168.226.176"
keyFile = "../data/rrg-bpet"
current_directory = os.getcwd()
key = os.path.join(current_directory, keyFile)

subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", "get_results.py",
    "ubuntu@{}:/home/ubuntu/".format(WATCHDOG_ADDRESS)])

COMMAND = 'python3 get_results.py'
subprocess.Popen(["ssh", "-i", key, 
                    "-o", "StrictHostKeyChecking=no", "ubuntu@%s" % WATCHDOG_ADDRESS, COMMAND],
                    shell=False,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE)
subprocess.run(['sleep', '3'])
subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", 
    "ubuntu@{}:/home/ubuntu/results.json".format(WATCHDOG_ADDRESS), current_directory])

subprocess.run(['sleep', '2'])

with open('results.json', 'r') as f:
    data = json.load(f)
    rows = data['results']



df_orig = pd.DataFrame(np.array(rows),
    columns=['Hostname', 'IP', 'NodeAddress', 'IsValidator'])

# sort dataframe
df = df_orig.copy()
idx = []
for name in df.Hostname:
    idx.append(int(name.split('-')[1]))
df['Index'] = idx
df = df.set_index(keys=df.Index).drop(labels='Index', axis=1).sort_index()
df

,Hostname,IP,NodeAddress,IsValidator
Index,,,,
1,besu-1,192.168.226.173,0x8b5a4561ccb044e345b20b56769a943444df490d,True
2,besu-2,192.168.226.86,0x1037869b64dabba6ef9ad5ac3f679ebc136d1b6c,True
3,besu-3,192.168.226.124,0xe7db4efc5c65f82287d045613557a8d5ed94dca9,True
4,besu-4,192.168.226.233,0xa422a3003a0a26c548be93d8de5804eaa3dd644f,True
5,besu-5,192.168.226.89,0x9a42b2652316cce01efaff91171430794e9ffc5e,True
6,besu-6,192.168.226.128,0x79753210c2c726d2eee7b79146ac8e5e3ff06687,True
7,besu-7,192.168.226.69,0x8b08afd8f8207de9e11c8bdaca083162ffbb0968,True
8,besu-8,192.168.226.52,0xe7e8b25500086ca761d9c0681144d409a0d3244a,True
